<a href="https://colab.research.google.com/github/Zabihuddin/data_analysis/blob/main/Copy_of_AI_Resume.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


#Complete Code

In [ ]:
import pandas as pd
import numpy as np
import spacy
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import FeatureUnion
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, classification_report, roc_auc_score
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from joblib import dump

nlp = spacy.load("en_core_web_sm")


In [ ]:
from google.colab import files
uploaded = files.upload()  # upload resume_job_pairs.csv

df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/tools & technique/resume & porfolio projects/AI-Powered Resume Screener /data.csv")  # make sure filename matches
df.head()


,Unnamed: 0,company,position,url,location,headquaters,employees,founded,industry,Job Description
0,1,Visual BI Solutions Inc,Graduate Intern (Summer 2017) - SAP BI / Big D...,https://www.glassdoor.com/partner/jobListing.h...,"Plano, TX","Plano, TX",51 to 200 employees,2010,Information Technology,"Location: Plano, TX or Oklahoma City, OK Dura..."
1,2,Jobvertise,Digital Marketing Manager,https://www.glassdoor.com/partner/jobListing.h...,"Dallas, TX","Berlin, Germany",1 to 50 employees,2011,Unknown,The Digital Marketing Manager is the front li...
2,3,Santander Consumer USA,"Manager, Pricing Management Information Systems",https://www.glassdoor.com/partner/jobListing.h...,"Dallas, TX","Dallas, TX",5001 to 10000 employees,1995,Finance,Summary of Responsibilities:The Manager Prici...
3,4,Federal Reserve Bank of Dallas,Treasury Services Analyst Internship,https://www.glassdoor.com/partner/jobListing.h...,"Dallas, TX","Dallas, TX",1001 to 5000 employees,1914,Finance,ORGANIZATIONAL SUMMARY: As part of the nati...
4,5,Aviall,"Intern, Sales Analyst",https://www.glassdoor.com/partner/jobListing.h...,"Dallas, TX","Dallas, TX",1001 to 5000 employees,Boeing,Subsidiary or Business Segment,Aviall is the world's largest provider of n...


In [ ]:


# Job dataset
df_jobs = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/tools & technique/resume & porfolio projects/AI-Powered Resume Screener /data.csv")

# Resume dataset
df_resumes = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/tools & technique/resume & porfolio projects/AI-Powered Resume Screener /UpdatedResumeDataSet.csv")

print("Jobs:", df_jobs.shape)
print("Resumes:", df_resumes.shape)

df_jobs.head()
df_resumes.head()





Jobs: (157, 10)
Resumes: (962, 2)


,Category,Resume
0,Data Science,Skills * Programming Languages: Python (pandas...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...
2,Data Science,"Areas of Interest Deep Learning, Control Syste..."
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab..."


In [ ]:

df_jobs = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/tools & technique/resume & porfolio projects/AI-Powered Resume Screener /data.csv")

# Resume dataset
df_resumes = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/tools & technique/resume & porfolio projects/AI-Powered Resume Screener /UpdatedResumeDataSet.csv")


In [ ]:
import re
import numpy as np
import pandas as pd

from sentence_transformers import SentenceTransformer, util
from sklearn.model_selection import GroupShuffleSplit
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score, f1_score, classification_report, roc_auc_score
)

# ============================================================
# 1. LOAD DATA
# ============================================================

jobs = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/tools & technique/resume & porfolio projects/AI-Powered Resume Screener /data.csv")
resumes = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/tools & technique/resume & porfolio projects/AI-Powered Resume Screener /UpdatedResumeDataSet.csv")


# Standardize column names (as you did before)
jobs = jobs.rename(columns={
    "position": "job_title",
    "Job Description": "job_text"
})
resumes = resumes.rename(columns={
    "Category": "category",
    "Resume": "resume_text"
})

# Create IDs to use for grouped splitting
jobs["job_id"] = np.arange(len(jobs))
resumes["resume_id"] = np.arange(len(resumes))

# ============================================================
# 2. SIMPLE TEXT CLEANING
#    (you can replace by your previous clean_text function)
# ============================================================

def clean_text(text):
    if pd.isna(text):
        return ""
    text = text.lower()
    text = re.sub(r"http\S+|www\.\S+", " ", text)
    text = re.sub(r"[^a-z0-9\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

resumes["resume_clean"] = resumes["resume_text"].apply(clean_text)
jobs["job_clean"]     = jobs["job_text"].apply(clean_text)

# ============================================================
# 3. BUILD RESUME–JOB PAIRS (RANDOM, NOT BASED ON CATEGORY)
# ============================================================

# How many jobs to sample per resume (you can tune this)
N_JOBS_PER_RESUME = 10

pair_rows = []

rng = np.random.default_rng(seed=42)

for _, r_row in resumes.iterrows():
    resume_id = r_row["resume_id"]
    resume_text = r_row["resume_clean"]

    # skip empty resumes
    if not isinstance(resume_text, str) or len(resume_text) < 20:
        continue

    # sample some jobs (without using category)
    sampled_jobs = jobs.sample(
        n=min(N_JOBS_PER_RESUME, len(jobs)),
        random_state=int(resume_id)  # stable sampling
    )

    for _, j_row in sampled_jobs.iterrows():
        pair_rows.append({
            "resume_id":  resume_id,
            "job_id":     j_row["job_id"],
            "resume_clean": resume_text,
            "job_clean":    j_row["job_clean"],
            "job_title":    j_row["job_title"]
        })

df_pairs = pd.DataFrame(pair_rows)
print("Number of pairs:", len(df_pairs))


Number of pairs: 9620


In [ ]:
# ============================================================
# 4. LABEL GENERATION VIA SEMANTIC SIMILARITY
# ============================================================

st_model = SentenceTransformer("all-MiniLM-L6-v2")

# Compute embeddings for each pair
resume_embs = st_model.encode(df_pairs["resume_clean"].tolist(), convert_to_tensor=True, show_progress_bar=True)
job_embs    = st_model.encode(df_pairs["job_clean"].tolist(), convert_to_tensor=True, show_progress_bar=True)

cos_sims = util.cos_sim(resume_embs, job_embs).diagonal().cpu().numpy()
df_pairs["similarity"] = cos_sims

# Choose thresholds (25th and 75th percentiles)
low_thr  = np.quantile(cos_sims, 0.25)
high_thr = np.quantile(cos_sims, 0.75)

def label_from_sim(s):
    if s >= high_thr:
        return 1
    elif s <= low_thr:
        return 0
    else:
        return -1  # ambiguous → ignore

df_pairs["label"] = df_pairs["similarity"].apply(label_from_sim)

# Keep only clear positives and negatives
df_pairs = df_pairs[df_pairs["label"] >= 0].reset_index(drop=True)
print(df_pairs["label"].value_counts())


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/301 [00:00<?, ?it/s]

Batches:   0%|          | 0/301 [00:00<?, ?it/s]

label
1    2405
0    2405
Name: count, dtype: int64


# Complete code

In [ ]:
# ============================================================
# 0) INSTALL & IMPORTS
# ============================================================

# If you're in Colab, uncomment:
# !pip install -q sentence-transformers

import re
import numpy as np
import pandas as pd

from sentence_transformers import SentenceTransformer, util

from sklearn.model_selection import GroupShuffleSplit
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score, f1_score, roc_auc_score,
    classification_report, confusion_matrix
)


# ============================================================
# 1) CONFIG
# ============================================================


JOBS_CSV_PATH    = "/content/drive/MyDrive/Colab Notebooks/tools & technique/resume & porfolio projects/AI-Powered Resume Screener /data.csv"
RESUMES_CSV_PATH = "/content/drive/MyDrive/Colab Notebooks/tools & technique/resume & porfolio projects/AI-Powered Resume Screener /UpdatedResumeDataSet.csv"

# Number of jobs to pair with each resume
N_JOBS_PER_RESUME = 10

# Max vocabulary size for TF-IDF
TFIDF_MAX_FEATURES = 5000


# ============================================================
# 2) LOAD DATA
# ============================================================

jobs = pd.read_csv(JOBS_CSV_PATH)
resumes = pd.read_csv(RESUMES_CSV_PATH)

# Standardize columns to common names
jobs = jobs.rename(columns={
    "position": "job_title",
    "Job Description": "job_text"
})
resumes = resumes.rename(columns={
    "Category": "category",
    "Resume": "resume_text"
})

print("Jobs sample:")
print(jobs.head(3), "\n")

print("Resumes sample:")
print(resumes.head(3), "\n")


# ============================================================
# 3) TEXT CLEANING
# ============================================================

def clean_text(text):
    if pd.isna(text):
        return ""
    text = text.lower()
    # Remove URLs
    text = re.sub(r"http\S+|www\.\S+", " ", text)
    # Remove non-alphanumeric except whitespace
    text = re.sub(r"[^a-z0-9\s]", " ", text)
    # Collapse spaces
    text = re.sub(r"\s+", " ", text).strip()
    return text

jobs["job_clean"]    = jobs["job_text"].apply(clean_text)
resumes["resume_clean"] = resumes["resume_text"].apply(clean_text)

# Add IDs to support grouped splitting
jobs["job_id"]    = np.arange(len(jobs))
resumes["resume_id"] = np.arange(len(resumes))

print("Cleaned jobs:")
print(jobs[["job_id", "job_title", "job_clean"]].head(3), "\n")

print("Cleaned resumes:")
print(resumes[["resume_id", "category", "resume_clean"]].head(3), "\n")


# ============================================================
# 4) BUILD RESUME–JOB PAIRS
#    (random pairing, not by category)
# ============================================================

pair_rows = []
rng = np.random.default_rng(seed=42)

for _, r_row in resumes.iterrows():
    resume_id   = r_row["resume_id"]
    resume_text = r_row["resume_clean"]

    # Skip very short or empty resumes
    if not isinstance(resume_text, str) or len(resume_text) < 20:
        continue

    sampled_jobs = jobs.sample(
        n=min(N_JOBS_PER_RESUME, len(jobs)),
        random_state=int(resume_id)  # stable sampling
    )

    for _, j_row in sampled_jobs.iterrows():
        pair_rows.append({
            "resume_id":    resume_id,
            "job_id":       j_row["job_id"],
            "resume_clean": resume_text,
            "job_clean":    j_row["job_clean"],
            "job_title":    j_row["job_title"]
        })

df_pairs = pd.DataFrame(pair_rows)
print("Number of resume–job pairs:", len(df_pairs))
print(df_pairs.head(), "\n")


# ============================================================
# 5) SEMANTIC SIMILARITY & LABEL GENERATION
#    Using SentenceTransformer as teacher
# ============================================================

print("Loading SentenceTransformer model...")
st_model = SentenceTransformer("all-MiniLM-L6-v2")

print("Encoding resumes...")
resume_embs = st_model.encode(
    df_pairs["resume_clean"].tolist(),
    convert_to_tensor=True,
    show_progress_bar=True
)

print("Encoding jobs...")
job_embs = st_model.encode(
    df_pairs["job_clean"].tolist(),
    convert_to_tensor=True,
    show_progress_bar=True
)

# Cosine similarity for each pair (diagonal of similarity matrix)
cos_sims = util.cos_sim(resume_embs, job_embs).diagonal().cpu().numpy()
df_pairs["similarity"] = cos_sims

# Use quartiles to define clear positive / negative pairs
low_thr  = np.quantile(cos_sims, 0.25)
high_thr = np.quantile(cos_sims, 0.75)

print(f"Similarity thresholds: low={low_thr:.3f}, high={high_thr:.3f}")

def label_from_sim(s):
    if s >= high_thr:
        return 1   # good match
    elif s <= low_thr:
        return 0   # poor match
    else:
        return -1  # ambiguous → drop

df_pairs["label"] = df_pairs["similarity"].apply(label_from_sim)

# Keep only clear positives (1) and negatives (0)
df_pairs = df_pairs[df_pairs["label"] >= 0].reset_index(drop=True)

print("Label counts:")
print(df_pairs["label"].value_counts(), "\n")


# ============================================================
# 6) BUILD COMBINED TEXT FEATURE
# ============================================================

def build_combined_text(row):
    return f"RESUME: {row['resume_clean']} JOB: {row['job_clean']}"

df_pairs["combined_text"] = df_pairs.apply(build_combined_text, axis=1)

X = df_pairs["combined_text"].values
y = df_pairs["label"].values
groups = df_pairs["resume_id"].values   # to avoid resume leakage


# ============================================================
# 7) GROUPED TRAIN–TEST SPLIT (no resume in both sets)
# ============================================================

from sklearn.model_selection import GroupShuffleSplit

gss = GroupShuffleSplit(
    n_splits=1,
    test_size=0.2,
    random_state=42
)
train_idx, test_idx = next(gss.split(X, y, groups=groups))

X_train, X_test = X[train_idx], X[test_idx]
y_train, y_test = y[train_idx], y[test_idx]

print("Train size:", len(X_train), "Test size:", len(X_test), "\n")


# ============================================================
# 8) TF-IDF + LOGISTIC REGRESSION
# ============================================================

vectorizer = TfidfVectorizer(
    max_features=TFIDF_MAX_FEATURES,
    ngram_range=(1, 2),
    min_df=2
)

X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec  = vectorizer.transform(X_test)

clf = LogisticRegression(
    max_iter=500,
    class_weight="balanced",
    solver="liblinear"
)

clf.fit(X_train_vec, y_train)


# ============================================================
# 9) EVALUATION
# ============================================================

y_pred = clf.predict(X_test_vec)
y_prob = clf.predict_proba(X_test_vec)[:, 1]

acc = accuracy_score(y_test, y_pred)
f1  = f1_score(y_test, y_pred)
roc = roc_auc_score(y_test, y_prob)

print(f"Accuracy: {acc:.3f}")
print(f"F1-score: {f1:.3f}")
print(f"ROC-AUC: {roc:.3f}\n")

print("Classification report:\n")
print(classification_report(y_test, y_pred))

print("Confusion matrix:\n")
print(confusion_matrix(y_test, y_pred))


# ============================================================
# 10) SIMPLE INTERPRETABILITY: KEYWORDS
# ============================================================

feature_names = np.array(vectorizer.get_feature_names_out())
coefs = clf.coef_[0]

# Top 30 positive features (indicate match)
top_pos_idx = np.argsort(coefs)[-30:]
# Top 30 negative features (indicate non-match)
top_neg_idx = np.argsort(coefs)[:30]

print("\nTop positive (match) features:\n")
for idx in reversed(top_pos_idx):
    print(f"{feature_names[idx]:25s}  coef={coefs[idx]:.3f}")

print("\nTop negative (non-match) features:\n")
for idx in top_neg_idx:
    print(f"{feature_names[idx]:25s}  coef={coefs[idx]:.3f}")


# ============================================================
# 11) NEXT-STEP UTILITIES: SCORING NEW PAIRS
# ============================================================

def score_resume_job(resume_text: str, job_text: str) -> float:
    """
    Use the trained TF-IDF + LogisticRegression model
    to score a new resume–job pair.

    Returns:
        Probability (0–1) that this is a good match (class 1).
    """
    r_clean = clean_text(resume_text)
    j_clean = clean_text(job_text)
    combined = f"RESUME: {r_clean} JOB: {j_clean}"
    vec = vectorizer.transform([combined])
    prob = clf.predict_proba(vec)[0, 1]
    return float(prob)

# Example usage:
example_resume = "Experienced data scientist with Python, machine learning, and deep learning."
example_job    = "We are hiring a data scientist with strong skills in Python, statistics, and ML."

print("\nExample scoring:")
print(score_resume_job(example_resume, example_job))


Jobs sample:
   Unnamed: 0                   company  \
0           1   Visual BI Solutions Inc   
1           2                Jobvertise   
2           3    Santander Consumer USA   

                                           job_title  \
0  Graduate Intern (Summer 2017) - SAP BI / Big D...   
1                          Digital Marketing Manager   
2    Manager, Pricing Management Information Systems   

                                                 url     location  \
0  https://www.glassdoor.com/partner/jobListing.h...    Plano, TX   
1  https://www.glassdoor.com/partner/jobListing.h...   Dallas, TX   
2  https://www.glassdoor.com/partner/jobListing.h...   Dallas, TX   

       headquaters                employees founded                industry  \
0        Plano, TX      51 to 200 employees    2010  Information Technology   
1  Berlin, Germany        1 to 50 employees    2011                 Unknown   
2       Dallas, TX  5001 to 10000 employees    1995                 Finance

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

Encoding resumes...


Batches:   0%|          | 0/301 [00:00<?, ?it/s]

Encoding jobs...


Batches:   0%|          | 0/301 [00:00<?, ?it/s]

Similarity thresholds: low=0.278, high=0.403
Label counts:
label
1    2405
0    2405
Name: count, dtype: int64 

Train size: 3852 Test size: 958 

Accuracy: 0.906
F1-score: 0.899
ROC-AUC: 0.973

Classification report:

              precision    recall  f1-score   support

           0       0.91      0.91      0.91       512
           1       0.90      0.90      0.90       446

    accuracy                           0.91       958
   macro avg       0.91      0.91      0.91       958
weighted avg       0.91      0.91      0.91       958

Confusion matrix:

[[466  46]
 [ 44 402]]

Top positive (match) features:

data                       coef=2.613
sap                        coef=2.511
management                 coef=2.397
with                       coef=2.385
months                     coef=2.140
project                    coef=2.114
python                     coef=2.009
sales                      coef=1.978
exprience                  coef=1.938
technical                  coef=1.824

In [ ]:
# ============================================================
# 12) SAVE MODEL & VECTORIZER FOR DEPLOYMENT
# ============================================================

import joblib

joblib.dump(vectorizer, "/content/drive/MyDrive/Colab Notebooks/tools & technique/resume & porfolio projects/AI-Powered Resume Screener /tfidf_vectorizer.pkl")
joblib.dump(clf, "/content/drive/MyDrive/Colab Notebooks/tools & technique/resume & porfolio projects/AI-Powered Resume Screener /resume_match_model.pkl")

print("Saved: tfidf_vectorizer.pkl and resume_match_model.pkl")


Saved: tfidf_vectorizer.pkl and resume_match_model.pkl


In [ ]:
!ls


drive  sample_data


In [ ]:
!pip install fastapi uvicorn pyngrok joblib scikit-learn


# complete code 1

In [ ]:
!pip install python-docx pdfplumber


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 82.6 MB/s eta 0:00:00


In [ ]:
import os
import re
import numpy as np
import pandas as pd
import pdfplumber
from docx import Document
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer

# ------------------------------------------------------------
# 1) Load trained model & vectorizer (if not already in memory)
# ------------------------------------------------------------
# If you are in the same notebook where you trained, comment these:
vectorizer = joblib.load("tfidf_vectorizer.pkl")
clf        = joblib.load("resume_match_model.pkl")


# ------------------------------------------------------------
# 2) Cleaning + scoring
# ------------------------------------------------------------
def clean_text(text: str) -> str:
    if text is None:
        return ""
    text = text.lower()
    text = re.sub(r"http\S+|www\.\S+", " ", text)
    text = re.sub(r"[^a-z0-9\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text


def score_resume_job(resume_text: str, job_text: str) -> float:
    """
    Use trained TF-IDF + LogisticRegression to score resume–job pair.
    Returns probability (0–1) of class 1 = good match.
    """
    r_clean = clean_text(resume_text)
    j_clean = clean_text(job_text)
    combined = f"RESUME: {r_clean} JOB: {j_clean}"
    vec = vectorizer.transform([combined])
    prob = clf.predict_proba(vec)[0, 1]
    return float(prob)


# ------------------------------------------------------------
# 3) Job description augmentation with keywords
# ------------------------------------------------------------
def augment_job_description(job_text: str, keywords=None) -> str:
    """
    Append extra keywords to the job description to emphasize
    important skills / technologies.
    """
    if not keywords:
        return job_text

    if isinstance(keywords, (list, tuple, set, np.ndarray)):
        kw_text = " ".join(str(k) for k in keywords)
    else:
        kw_text = str(keywords)

    augmented = job_text.strip() + "\n\nExtra keywords: " + kw_text
    return augmented


# ------------------------------------------------------------
# 4) Simple keyword extraction from job description
#    (helps you decide which keywords to add)
# ------------------------------------------------------------
def extract_job_keywords(job_text: str, top_n: int = 20) -> list[str]:
    """
    Very simple TF-IDF-based keyword extractor for a single job description.
    Returns a list of top_n keyword candidates.
    """
    cleaned = clean_text(job_text)
    if not cleaned:
        return []

    # Fit TF-IDF on this single document
    tfidf = TfidfVectorizer(
        max_features=1000,
        ngram_range=(1, 2),
        min_df=1,
        stop_words="english"
    )
    X = tfidf.fit_transform([cleaned])
    scores = X.toarray()[0]
    feature_names = np.array(tfidf.get_feature_names_out())

    # Sort by TF-IDF descending
    idx = np.argsort(scores)[::-1]
    top_idx = idx[:top_n]
    keywords = feature_names[top_idx].tolist()
    return keywords


# ------------------------------------------------------------
# 5) Load resume text from .txt, .pdf, .docx
# ------------------------------------------------------------
def load_resume_text(path: str) -> str:
    """
    Load resume text from a file.
    Supports: .txt, .pdf, .docx
    """
    ext = os.path.splitext(path)[1].lower()

    if ext == ".txt":
        with open(path, "r", encoding="utf-8", errors="ignore") as f:
            return f.read()

    elif ext == ".pdf":
        text_chunks = []
        with pdfplumber.open(path) as pdf:
            for page in pdf.pages:
                text_chunks.append(page.extract_text() or "")
        return "\n".join(text_chunks)

    elif ext == ".docx":
        doc = Document(path)
        text_chunks = [para.text for para in doc.paragraphs]
        return "\n".join(text_chunks)

    else:
        raise ValueError(f"Unsupported file type: {ext}. Use .txt, .pdf, or .docx.")


# ------------------------------------------------------------
# 6) Recursive folder scanning with optional category filter
# ------------------------------------------------------------
def score_resumes_in_folder(
    resume_root: str,
    job_description: str,
    keywords=None,
    categories: list[str] | None = None,
    threshold: float = 0.5,
    save_csv: str | None = None
) -> pd.DataFrame:
    """
    Recursively scan resume_root for .txt/.pdf/.docx files,
    score each resume vs job_description (+ keywords),
    and label as 'relevant' or 'irrelevant'.

    Args:
        resume_root: root folder containing category subfolders
                     e.g. INFORMATION-TECHNOLOGY, FINANCE, etc.
        job_description: raw job description string
        keywords: extra keywords to append to job description
        categories: optional list of category folder names to include
                    e.g. ["INFORMATION-TECHNOLOGY", "ENGINEERING"]
                    If None → use all categories.
        threshold: probability threshold for relevance
        save_csv: optional path to save results

    Returns:
        DataFrame with columns:
        ['filename', 'path', 'category', 'probability', 'label']
    """
    job_aug = augment_job_description(job_description, keywords)
    results = []

    print(f"\nScanning root folder recursively: {resume_root}")
    if categories is not None:
        print("Filtering to categories:", categories)

    for root, dirs, files in os.walk(resume_root):
        # Infer category from first-level subfolder (if applicable)
        # e.g. /root/INFORMATION-TECHNOLOGY/resume1.txt → category = INFORMATION-TECHNOLOGY
        rel = os.path.relpath(root, resume_root)
        parts = rel.split(os.sep)
        category = parts[0] if parts[0] != "." else None

        # If categories filter is given, skip roots not matching
        if categories is not None and category is not None and category not in categories:
            continue

        for fname in files:
            fpath = os.path.join(root, fname)
            ext = os.path.splitext(fname)[1].lower()

            if ext not in [".txt", ".pdf", ".docx"]:
                print(f"Skipping {fname}: unsupported file type {ext}")
                continue

            try:
                resume_text = load_resume_text(fpath)
            except Exception as e:
                print(f"Skipping {fname}: {e}")
                continue

            if not resume_text.strip():
                print(f"Skipping {fname}: empty/unreadable")
                continue

            prob = score_resume_job(resume_text, job_aug)
            label = "relevant" if prob >= threshold else "irrelevant"

            results.append({
                "filename": fname,
                "path": fpath,
                "category": category,
                "probability": prob,
                "label": label
            })

    if not results:
        print("\nNo resume files were found in the specified folders.")
        return pd.DataFrame(columns=["filename", "path", "category", "probability", "label"])

    df = pd.DataFrame(results)
    df = df.sort_values("probability", ascending=False).reset_index(drop=True)

    if save_csv:
        df.to_csv(save_csv, index=False)
        print(f"Saved results to: {save_csv}")

    return df



resume_root = "/content/drive/MyDrive/Colab Notebooks/tools & technique/resume & porfolio projects/AI-Powered Resume Screener /data/data"

job_description = """
We are looking for a Data Scientist to join our analytics team.

Responsibilities:
- Build and validate predictive models for customer behavior and business KPIs
- Perform data cleaning, feature engineering, and model evaluation using Python
- Deploy models as APIs in production
- Communicate results through dashboards and visualizations

Required skills:
- Strong experience with Python (pandas, NumPy, scikit-learn)
- Experience with SQL and large datasets
- Familiarity with cloud data platforms (AWS, GCP or Azure)
"""

# 1) Let the code suggest keywords for you
suggested_keywords = extract_job_keywords(job_description, top_n=20)
print("Suggested job keywords:", suggested_keywords)

# You can then manually pick / modify:
job_keywords = ["python", "pandas", "scikit-learn", "sql", "aws", "machine learning", "xgboost"]

# 2) Score all resumes, but only IT + ENGINEERING categories
df_scores = score_resumes_in_folder(
    resume_root=resume_root,
    job_description=job_description,
    keywords=job_keywords,
    categories=["INFORMATION-TECHNOLOGY", "ENGINEERING"],  # or None for all
    threshold=0.5,
    save_csv="resume_screening_results.csv"
)

print("\nBatch screening results:")
print(df_scores.head())

if not df_scores.empty:
    relevant   = df_scores[df_scores["label"] == "relevant"]
    irrelevant = df_scores[df_scores["label"] == "irrelevant"]

    print("\nMost relevant resumes:")
    print(relevant[["filename", "category", "probability"]].head())

    print("\nIrrelevant resumes:")
    print(irrelevant[["filename", "category", "probability"]].head())


Suggested job keywords: ['data', 'python', 'models', 'experience', 'visualizations', 'validate', 'team responsibilities', 'validate predictive', 'using python', 'using', 'sql large', 'sql', 'skills strong', 'skills', 'scikit learn', 'team', 'strong experience', 'strong', 'scientist', 'results dashboards']

Scanning root folder recursively: /content/drive/MyDrive/Colab Notebooks/tools & technique/resume & porfolio projects/AI-Powered Resume Screener /data/data
Filtering to categories: ['INFORMATION-TECHNOLOGY', 'ENGINEERING']
Saved results to: resume_screening_results.csv

Batch screening results:
       filename                                               path  \
0  30223363.pdf  /content/drive/MyDrive/Colab Notebooks/tools &...   
1  27152464.pdf  /content/drive/MyDrive/Colab Notebooks/tools &...   
2  52618188.pdf  /content/drive/MyDrive/Colab Notebooks/tools &...   
3  37242217.pdf  /content/drive/MyDrive/Colab Notebooks/tools &...   
4  13405733.pdf  /content/drive/MyDrive/Colab 

# Complete code 2

In [ ]:
!pip install fastapi uvicorn pdfplumber python-docx pillow pytesseract transformers accelerate joblib pandas


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 58.9 MB/s eta 0:00:00


In [ ]:
# ============================================================
# AI Resume Matcher - Training Script (multi-format resumes)
# ============================================================

import os
import io
import re
import numpy as np
import pandas as pd

import pdfplumber
from docx import Document
from PIL import Image
import pytesseract

from sentence_transformers import SentenceTransformer, util

from sklearn.model_selection import GroupShuffleSplit
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score, f1_score, roc_auc_score,
    classification_report, confusion_matrix
)
import joblib


# ============================================================
# 0) CONFIG
# ============================================================

# ---- (A) Job descriptions CSV (same as before) ----
JOBS_CSV_PATH = (
    "/content/drive/MyDrive/Colab Notebooks/tools & technique/"
    "resume & porfolio projects/AI-Powered Resume Screener /data.csv"
)

# ---- (B) Resumes: choose ONE mode ----
USE_RESUME_FOLDER = True   # <--- set True to read PDF/DOCX/IMG/TXT/CSV resumes
RESUMES_FOLDER = (
    "/content/drive/MyDrive/Colab Notebooks/tools & technique/resume & porfolio projects/AI-Powered Resume Screener /data/data"
)

# If you still want to use the old CSV-based dataset, set:
# USE_RESUME_FOLDER = False
RESUMES_CSV_PATH = (
    "/content/drive/MyDrive/Colab Notebooks/tools & technique/"
    "resume & porfolio projects/AI-Powered Resume Screener /UpdatedResumeDataSet.csv"
)

# Number of jobs to pair with each resume
N_JOBS_PER_RESUME = 10

# Max vocabulary size for TF-IDF
TFIDF_MAX_FEATURES = 5000

# Where to save the trained model
VEC_OUT = "/content/drive/MyDrive/Colab Notebooks/tools & technique/resume & porfolio projects/AI-Powered Resume Screener /tfidf_vectorizer.pkl"
CLF_OUT = "/content/drive/MyDrive/Colab Notebooks/tools & technique/resume & porfolio projects/AI-Powered Resume Screener /resume_match_model.pkl"
PAIRS_OUT = "/content/drive/MyDrive/Colab Notebooks/tools & technique/resume & porfolio projects/AI-Powered Resume Screener /training_pairs.csv"   # to support later re-training


# ============================================================
# 1) TEXT CLEANING
# ============================================================

def clean_text(text):
    if text is None or (isinstance(text, float) and np.isnan(text)):
        return ""
    text = str(text).lower()
    # Remove URLs
    text = re.sub(r"http\S+|www\.\S+", " ", text)
    # Remove non-alphanumeric except whitespace
    text = re.sub(r"[^a-z0-9\s]", " ", text)
    # Collapse spaces
    text = re.sub(r"\s+", " ", text).strip()
    return text


# ============================================================
# 2) FILE-LEVEL TEXT EXTRACTION (PDF, DOCX, TXT, CSV, IMG)
# ============================================================

def extract_text_from_path(path: str) -> str:
    """
    Universal text extractor for resumes.
    Supports:
      - .txt
      - .pdf
      - .docx
      - .csv  (concatenate text column(s))
      - .jpg/.jpeg/.png (OCR via Tesseract)
    """
    ext = os.path.splitext(path)[1].lower()

    # TXT
    if ext == ".txt":
        try:
            with open(path, "r", encoding="utf-8", errors="ignore") as f:
                return f.read()
        except Exception:
            return ""

    # PDF
    if ext == ".pdf":
        try:
            text_chunks = []
            with pdfplumber.open(path) as pdf:
                for page in pdf.pages:
                    text_chunks.append(page.extract_text() or "")
            return "\n".join(text_chunks)
        except Exception:
            return ""

    # DOCX
    if ext == ".docx":
        try:
            doc = Document(path)
            return "\n".join(p.text for p in doc.paragraphs)
        except Exception:
            return ""

    # CSV (for resume-like data)
    if ext == ".csv":
        try:
            df = pd.read_csv(path)
            # Try common text columns
            candidate_cols = ["resume", "Resume", "resume_text", "text", "content"]
            for c in candidate_cols:
                if c in df.columns:
                    return "\n\n".join(df[c].astype(str).tolist())
            # Fallback: entire CSV as string
            return df.to_string()
        except Exception:
            return ""

    # Image (OCR)
    if ext in [".jpg", ".jpeg", ".png"]:
        try:
            img = Image.open(path)
            text = pytesseract.image_to_string(img)
            return text
        except Exception:
            return ""

    # Unsupported extension
    return ""


# ============================================================
# 3) LOAD JOB DATA
# ============================================================

jobs = pd.read_csv(JOBS_CSV_PATH)

# Standardize columns to common names
jobs = jobs.rename(columns={
    "position": "job_title",
    "Job Description": "job_text"
})

jobs["job_clean"] = jobs["job_text"].apply(clean_text)
jobs["job_id"] = np.arange(len(jobs))

print("Jobs sample:")
print(jobs[["job_id", "job_title", "job_clean"]].head(3), "\n")


# ============================================================
# 4) LOAD RESUME DATA (CSV OR FOLDER OF FILES)
# ============================================================

if USE_RESUME_FOLDER:
    resume_records = []
    idx = 0
    for root, _, files in os.walk(RESUMES_FOLDER):
        for fname in files:
            path = os.path.join(root, fname)
            ext = os.path.splitext(fname)[1].lower()
            if ext not in [".pdf", ".docx", ".txt", ".csv", ".jpg", ".jpeg", ".png"]:
                # skip unsupported/reserved files
                continue

            raw_text = extract_text_from_path(path)
            if not raw_text or len(raw_text.strip()) < 20:
                continue

            resume_records.append({
                "resume_id": idx,
                "filename": fname,
                "path": path,
                "resume_text": raw_text
            })
            idx += 1

    resumes = pd.DataFrame(resume_records)
    if resumes.empty:
        raise RuntimeError("No valid resume text found in folder. Check RESUMES_FOLDER.")

    resumes["category"] = "UNKNOWN"
    resumes["resume_clean"] = resumes["resume_text"].apply(clean_text)

else:
    resumes = pd.read_csv(RESUMES_CSV_PATH)
    resumes = resumes.rename(columns={
        "Category": "category",
        "Resume": "resume_text"
    })
    resumes["resume_clean"] = resumes["resume_text"].apply(clean_text)
    resumes["resume_id"] = np.arange(len(resumes))

print("Resumes sample:")
print(resumes[["resume_id", "category", "resume_clean"]].head(3), "\n")


# ============================================================
# 5) BUILD RESUME–JOB PAIRS
#    (random pairing, not by category)
# ============================================================

pair_rows = []
rng = np.random.default_rng(seed=42)

for _, r_row in resumes.iterrows():
    resume_id = r_row["resume_id"]
    resume_text = r_row["resume_clean"]

    # Skip very short or empty resumes
    if not isinstance(resume_text, str) or len(resume_text) < 20:
        continue

    sampled_jobs = jobs.sample(
        n=min(N_JOBS_PER_RESUME, len(jobs)),
        random_state=int(resume_id)  # stable sampling
    )

    for _, j_row in sampled_jobs.iterrows():
        pair_rows.append({
            "resume_id": resume_id,
            "job_id": j_row["job_id"],
            "resume_clean": resume_text,
            "job_clean": j_row["job_clean"],
            "job_title": j_row["job_title"]
        })

df_pairs = pd.DataFrame(pair_rows)
print("Number of resume–job pairs:", len(df_pairs))
print(df_pairs.head(), "\n")


# ============================================================
# 6) SEMANTIC SIMILARITY & LABEL GENERATION
#    Using SentenceTransformer as teacher
# ============================================================

print("Loading SentenceTransformer model...")
st_model = SentenceTransformer("all-MiniLM-L6-v2")

print("Encoding resumes...")
resume_embs = st_model.encode(
    df_pairs["resume_clean"].tolist(),
    convert_to_tensor=True,
    show_progress_bar=True
)

print("Encoding jobs...")
job_embs = st_model.encode(
    df_pairs["job_clean"].tolist(),
    convert_to_tensor=True,
    show_progress_bar=True
)

# Cosine similarity for each pair (diagonal of similarity matrix)
cos_sims = util.cos_sim(resume_embs, job_embs).diagonal().cpu().numpy()
df_pairs["similarity"] = cos_sims

# Use quartiles to define clear positive / negative pairs
low_thr = np.quantile(cos_sims, 0.25)
high_thr = np.quantile(cos_sims, 0.75)

print(f"Similarity thresholds: low={low_thr:.3f}, high={high_thr:.3f}")

def label_from_sim(s):
    if s >= high_thr:
        return 1   # good match
    elif s <= low_thr:
        return 0   # poor match
    else:
        return -1  # ambiguous → drop

df_pairs["label"] = df_pairs["similarity"].apply(label_from_sim)

# Keep only clear positives (1) and negatives (0)
df_pairs = df_pairs[df_pairs["label"] >= 0].reset_index(drop=True)

print("Label counts:")
print(df_pairs["label"].value_counts(), "\n")


# ============================================================
# 7) BUILD COMBINED TEXT FEATURE
# ============================================================

def build_combined_text(row):
    return f"RESUME: {row['resume_clean']} JOB: {row['job_clean']}"

df_pairs["combined_text"] = df_pairs.apply(build_combined_text, axis=1)

X = df_pairs["combined_text"].values
y = df_pairs["label"].values
groups = df_pairs["resume_id"].values   # to avoid resume leakage


# ============================================================
# 8) GROUPED TRAIN–TEST SPLIT (no resume in both sets)
# ============================================================

gss = GroupShuffleSplit(
    n_splits=1,
    test_size=0.2,
    random_state=42
)
train_idx, test_idx = next(gss.split(X, y, groups=groups))

X_train, X_test = X[train_idx], X[test_idx]
y_train, y_test = y[train_idx], y[test_idx]

print("Train size:", len(X_train), "Test size:", len(X_test), "\n")


# ============================================================
# 9) TF-IDF + LOGISTIC REGRESSION
# ============================================================

vectorizer = TfidfVectorizer(
    max_features=TFIDF_MAX_FEATURES,
    ngram_range=(1, 2),
    min_df=2
)

X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

clf = LogisticRegression(
    max_iter=500,
    class_weight="balanced",
    solver="liblinear"
)

clf.fit(X_train_vec, y_train)


# ============================================================
# 10) EVALUATION
# ============================================================

y_pred = clf.predict(X_test_vec)
y_prob = clf.predict_proba(X_test_vec)[:, 1]

acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc = roc_auc_score(y_test, y_prob)

print(f"Accuracy: {acc:.3f}")
print(f"F1-score: {f1:.3f}")
print(f"ROC-AUC: {roc:.3f}\n")

print("Classification report:\n")
print(classification_report(y_test, y_pred))

print("Confusion matrix:\n")
print(confusion_matrix(y_test, y_pred))


# ============================================================
# 11) SIMPLE INTERPRETABILITY: KEYWORDS
# ============================================================

feature_names = np.array(vectorizer.get_feature_names_out())
coefs = clf.coef_[0]

# Top 30 positive features (indicate match)
top_pos_idx = np.argsort(coefs)[-30:]
# Top 30 negative features (indicate non-match)
top_neg_idx = np.argsort(coefs)[:30]

print("\nTop positive (match) features:\n")
for idx in reversed(top_pos_idx):
    print(f"{feature_names[idx]:25s}  coef={coefs[idx]:.3f}")

print("\nTop negative (non-match) features:\n")
for idx in top_neg_idx:
    print(f"{feature_names[idx]:25s}  coef={coefs[idx]:.3f}")


# ============================================================
# 12) SAVE MODEL + TRAINING PAIRS (for future self-learning)
# ============================================================

# Save training pairs so you can later extend or retrain
df_pairs[["combined_text", "label", "resume_id", "job_id"]].to_csv(
    PAIRS_OUT,
    index=False
)

joblib.dump(vectorizer, VEC_OUT)
joblib.dump(clf, CLF_OUT)

print(f"\nSaved vectorizer to {VEC_OUT}")
print(f"Saved classifier to {CLF_OUT}")
print(f"Saved training pairs to {PAIRS_OUT}")


# ============================================================
# 13) UTILITY: SCORE NEW PAIR (same as in API)
# ============================================================

def score_resume_job(resume_text: str, job_text: str) -> float:
    """
    Use the trained TF-IDF + LogisticRegression model
    to score a new resume–job pair.

    Returns:
        Probability (0–1) that this is a good match (class 1).
    """
    r_clean = clean_text(resume_text)
    j_clean = clean_text(job_text)
    combined = f"RESUME: {r_clean} JOB: {j_clean}"
    vec = vectorizer.transform([combined])
    prob = clf.predict_proba(vec)[0, 1]
    return float(prob)


print("\nExample scoring:")
example_resume = "Experienced data scientist with Python, machine learning, and deep learning."
example_job = "We are hiring a data scientist with strong skills in Python, statistics, and ML."
print(score_resume_job(example_resume, example_job))


Jobs sample:
   job_id                                          job_title  \
0       0  Graduate Intern (Summer 2017) - SAP BI / Big D...   
1       1                          Digital Marketing Manager   
2       2    Manager, Pricing Management Information Systems   

                                           job_clean  
0  location plano tx or oklahoma city ok duration...  
1  the digital marketing manager is the front lin...  
2  summary of responsibilities the manager pricin...   

Resumes sample:
   resume_id category                                       resume_clean
0          0  UNKNOWN  substitute teacher summary receiving an educat...
1          1  UNKNOWN  history teacher experience history teacher 08 ...
2          2  UNKNOWN  tw substitute teacher experience company name ... 

Number of resume–job pairs: 24830
   resume_id  job_id                                       resume_clean  \
0          0       7  substitute teacher summary receiving an educat...   
1          0  

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

Encoding resumes...


Batches:   0%|          | 0/776 [00:00<?, ?it/s]

Encoding jobs...


Batches:   0%|          | 0/776 [00:00<?, ?it/s]

Similarity thresholds: low=0.285, high=0.413
Label counts:
label
0    6208
1    6208
Name: count, dtype: int64 

Train size: 9966 Test size: 2450 

Accuracy: 0.824
F1-score: 0.835
ROC-AUC: 0.903

Classification report:

              precision    recall  f1-score   support

           0       0.81      0.81      0.81      1149
           1       0.83      0.84      0.83      1301

    accuracy                           0.82      2450
   macro avg       0.82      0.82      0.82      2450
weighted avg       0.82      0.82      0.82      2450

Confusion matrix:

[[ 931  218]
 [ 213 1088]]

Top positive (match) features:

business                   coef=4.398
management                 coef=3.894
project                    coef=3.362
sales                      coef=3.315
to                         coef=2.416
analysis                   coef=2.362
support                    coef=2.079
marketing                  coef=2.070
client                     coef=2.016
office                     coef=

In [ ]:
%%writefile api1.py
# ============================================================
# api1.py
# AI Resume Recommendation System Backend
# - Uses tfidf_vectorizer.pkl + resume_match_model.pkl
#   saved by the training script you shared
# - Universal file reader (PDF, DOCX, TXT, CSV, JPG/PNG OCR)
# - TF-IDF + LogisticRegression matcher
# - AI summary (general, non-domain-specific)
# - Light resume parsing -> structured JSON
# - Job keyword extraction + keyword-based adjustment of score
# - FastAPI endpoint: /analyze_files
# ============================================================

import io
import os
import re
import logging
from typing import List, Dict, Any

import pdfplumber
import pandas as pd
from docx import Document
from PIL import Image
import pytesseract

import joblib
from fastapi import FastAPI, UploadFile, File, Form, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import HTMLResponse
from pydantic import BaseModel

from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import pipeline

# ------------------------------------------------------------
# 0) Logging
# ------------------------------------------------------------
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# ------------------------------------------------------------
# 1) Load trained model artifacts (must match training script)
# ------------------------------------------------------------

BASE_PATH = (
    "/content/drive/MyDrive/Colab Notebooks/tools & technique/"
    "resume & porfolio projects/AI-Powered Resume Screener "
)  # note the trailing space – same as in training script

VEC_PATH = BASE_PATH + "/tfidf_vectorizer.pkl"
CLF_PATH = BASE_PATH + "/resume_match_model.pkl"
PAIRS_PATH = BASE_PATH + "/training_pairs.csv"  # optional

if not os.path.exists(VEC_PATH):
    raise FileNotFoundError(f"Vectorizer not found at {VEC_PATH}")
if not os.path.exists(CLF_PATH):
    raise FileNotFoundError(f"Classifier not found at {CLF_PATH}")

vectorizer: TfidfVectorizer = joblib.load(VEC_PATH)
clf = joblib.load(CLF_PATH)

training_pairs = None
if os.path.exists(PAIRS_PATH):
    try:
        training_pairs = pd.read_csv(PAIRS_PATH)
        logger.info(f"Loaded training pairs from: {PAIRS_PATH}")
    except Exception as e:
        logger.warning(f"Failed to load training_pairs.csv: {e}")

# ------------------------------------------------------------
# 2) Cleaning + scoring (must match training script)
# ------------------------------------------------------------

def clean_text(text: str) -> str:
    """Same cleaning as in training script."""
    if text is None:
        return ""
    # training script converts to str and lowercases
    text = str(text).lower()
    text = re.sub(r"http\S+|www\.\S+", " ", text)
    text = re.sub(r"[^a-z0-9\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text


def score_resume_job(resume_text: str, job_text: str) -> float:
    """
    Scoring function identical to training script:
    build 'RESUME: ... JOB: ...', transform with TF-IDF,
    and predict probability of class 1.
    """
    r_clean = clean_text(resume_text)
    j_clean = clean_text(job_text)
    combined = f"RESUME: {r_clean} JOB: {j_clean}"
    vec = vectorizer.transform([combined])
    prob = clf.predict_proba(vec)[0, 1]
    return float(prob)


def combine_scores(model_prob: float, match_ratio: float) -> float:
    """
    Combine ML model probability and keyword match ratio
    into a final score.

    - If keywords do not match (match_ratio ~ 0), final score is penalised.
    - If both model_prob and match_ratio are high, final score increases.
    """
    alpha = 0.7  # weight for model probability
    beta = 0.3   # weight for keyword match ratio

    final_prob = alpha * model_prob + beta * match_ratio

    # Penalise hard if there is no keyword overlap at all
    if match_ratio == 0.0:
        final_prob *= 0.3

    return float(max(0.0, min(1.0, final_prob)))


# ------------------------------------------------------------
# 3) Job keyword extraction (frequency-based)
# ------------------------------------------------------------
STOPWORDS = {
    "the", "a", "an", "and", "or", "for", "with", "to", "in", "on", "of",
    "at", "by", "is", "are", "be", "as", "that", "this", "we", "our", "you",
    "from", "will", "shall", "they", "their", "your", "his", "her", "it",
    "its", "have", "has", "had", "about", "into", "such", "also", "etc",
    "must", "should", "can", "may", "per", "based"
}


def extract_keywords(text: str, top_n: int = 15) -> List[str]:
    """Auto-suggested job keywords from the description."""
    cleaned = clean_text(text)
    tokens = [t for t in cleaned.split() if t not in STOPWORDS and len(t) > 2]
    if not tokens:
        return []
    s = pd.Series(tokens)
    return s.value_counts().head(top_n).index.tolist()


# ------------------------------------------------------------
# 4) Summarisation model
# ------------------------------------------------------------
try:
    summarizer = pipeline(
        "summarization",
        model="sshleifer/distilbart-cnn-12-6"
    )
    logger.info("Loaded summarisation model: sshleifer/distilbart-cnn-12-6")
except Exception as e:
    summarizer = None
    logger.warning(f"Could not load summarisation model. Fallback will be used. Error: {e}")


def generate_summary(text: str) -> str:
    """
    Generate a short, general summary of a resume
    (~one page of important information).
    """
    text = (text or "").strip()
    if not text:
        return ""

    if summarizer is not None and len(text) > 100:
        sample = text[:2500]
        try:
            out = summarizer(
                sample,
                max_length=180,
                min_length=60,
                do_sample=False
            )
            summary = out[0]["summary_text"].strip()
            if summary:
                return summary
        except Exception as e:
            logger.warning(f"Summarisation failed, using fallback. Error: {e}")

    preview = text[:1200]
    sentences = re.split(r'(?<=[.!?])\s+', preview)
    sentences = [s.strip() for s in sentences if s.strip()]
    if len(sentences) >= 3:
        return " ".join(sentences[:3])
    elif len(sentences) >= 1:
        return " ".join(sentences[:2])
    return preview[:400]


# ------------------------------------------------------------
# 5) Universal file text extraction (UploadFile)
# ------------------------------------------------------------
def extract_text_from_upload(upload: UploadFile) -> str:
    """
    Extract text from an uploaded resume file.
    Supported:
      - .txt
      - .pdf
      - .docx
      - .csv (concatenate text column(s))
      - .jpg/.jpeg/.png (OCR via Tesseract)
    """
    filename = upload.filename or "file"
    ext = filename.split(".")[-1].lower()
    data = upload.file.read()

    if ext == "txt":
        try:
            return data.decode("utf-8", errors="ignore")
        except Exception as e:
            logger.warning(f"Failed to read TXT {filename}: {e}")
            return ""

    if ext == "pdf":
        try:
            text_chunks = []
            with pdfplumber.open(io.BytesIO(data)) as pdf:
                for page in pdf.pages:
                    text_chunks.append(page.extract_text() or "")
            return "\n".join(text_chunks)
        except Exception as e:
            logger.warning(f"Failed to read PDF {filename}: {e}")
            return ""

    if ext == "docx":
        try:
            doc = Document(io.BytesIO(data))
            return "\n".join(p.text for p in doc.paragraphs)
        except Exception as e:
            logger.warning(f"Failed to read DOCX {filename}: {e}")
            return ""

    if ext == "csv":
        try:
            df = pd.read_csv(io.BytesIO(data))
            candidate_cols = ["resume", "Resume", "resume_text", "text", "content"]
            for c in candidate_cols:
                if c in df.columns:
                    return "\n\n".join(df[c].astype(str).tolist())
            return df.to_string()
        except Exception as e:
            logger.warning(f"Failed to read CSV {filename}: {e}")
            return ""

    if ext in {"jpg", "jpeg", "png"}:
        try:
            img = Image.open(io.BytesIO(data))
            return pytesseract.image_to_string(img)
        except Exception as e:
            logger.warning(f"OCR failed for {filename}: {e}")
            return ""

    logger.warning(f"Unsupported file type: {filename}")
    return ""


# ------------------------------------------------------------
# 6) Light resume parsing -> structured info
# ------------------------------------------------------------
def extract_structured_info(text: str) -> Dict[str, Any]:
    """
    Light structured parsing:
      - name, email, phone, LinkedIn, GitHub
      - skills (top tokens)
      - education snippets
      - experience snippets
      - preview text
    """
    info: Dict[str, Any] = {}
    raw = text or ""

    lines = raw.strip().splitlines()
    first_line = lines[0].strip() if lines else ""

    info["name"] = re.sub(r"[^A-Za-z\s]", "", first_line).strip() or None

    email_match = re.search(
        r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}",
        raw
    )
    info["email"] = email_match.group(0) if email_match else None

    phone_match = re.search(r"\+?\d[\d\s\-]{8,15}", raw)
    info["phone"] = phone_match.group(0) if phone_match else None

    linkedin_match = re.search(r"(https?://)?(www\.)?linkedin\.com/[^\s]+", raw)
    github_match = re.search(r"(https?://)?(www\.)?github\.com/[^\s]+", raw)
    info["linkedin"] = linkedin_match.group(0) if linkedin_match else None
    info["github"] = github_match.group(0) if github_match else None

    cleaned = clean_text(raw)
    tokens = cleaned.split()
    if tokens:
        s = pd.Series(tokens)
        top_tokens = s.value_counts().head(20).index.tolist()
    else:
        top_tokens = []
    info["skills"] = top_tokens

    exp_snips = re.findall(
        r"\d+\+?\s+years? of [^.;,\n]+",
        raw,
        flags=re.IGNORECASE
    )
    info["experience"] = list(dict.fromkeys(exp_snips))

    edu_snips = re.findall(
        r"(B\.?S\.?|BSc|B\.?E\.?|Bachelor|M\.?S\.?|MSc|Master|PhD|Doctorate)[^,\n]*",
        raw,
        flags=re.IGNORECASE
    )
    info["education"] = list(dict.fromkeys(edu_snips))

    info["preview"] = raw.strip()[:1500]

    return info


# ------------------------------------------------------------
# 7) FastAPI app & schemas
# ------------------------------------------------------------
app = FastAPI(
    title="AI Resume Recommendation System",
    description="Scores resumes vs job description using your trained TF-IDF + LogisticRegression model.",
    version="2.0.0"
)

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)


class FileResult(BaseModel):
    filename: str
    probability: float          # final combined probability
    label: str                  # relevant / irrelevant
    summary: str
    parsed: Dict[str, Any]      # structured info + diagnostics


class AnalyzeResponse(BaseModel):
    total_files: int
    relevant_count: int
    irrelevant_count: int
    results: List[FileResult]
    job_keywords: List[str]


# ------------------------------------------------------------
# 8) Health + root
# ------------------------------------------------------------
@app.get("/")
def root():
    return {
        "message": "AI Resume Recommendation System API",
        "endpoints": ["/health", "/analyze_files", "/app", "/docs"]
    }


@app.get("/health")
def health():
    return {"status": "ok"}


# ------------------------------------------------------------
# 9) Main endpoint: /analyze_files
# ------------------------------------------------------------
@app.post("/analyze_files", response_model=AnalyzeResponse)
async def analyze_files(
    job: str = Form(...),
    threshold: float = Form(0.5),
    files: List[UploadFile] = File(...)
):
    """
    For each uploaded resume:
      - Extract text
      - Compute model probability (semantic similarity)
      - Compute keyword match ratio (job vs resume)
      - Combine into final probability
      - Produce summary and structured info
    """
    job = (job or "").strip()
    if not job:
        raise HTTPException(status_code=400, detail="'job' must be non-empty.")

    if not (0.0 <= threshold <= 1.0):
        raise HTTPException(status_code=400, detail="'threshold' must be between 0 and 1.")

    if not files:
        raise HTTPException(status_code=400, detail="At least one file must be uploaded.")

    job_keywords = extract_keywords(job, top_n=15)
    results: List[FileResult] = []
    processed_files = 0

    for upload in files:
        text = extract_text_from_upload(upload)
        if not text or not text.strip():
            logger.warning(f"Skipping empty/unreadable file: {upload.filename}")
            continue

        processed_files += 1

        # 1) Model probability
        model_prob = score_resume_job(text, job)

        # 2) Keyword match
        resume_tokens = set(clean_text(text).split())
        matched_keywords = [kw for kw in job_keywords if kw in resume_tokens]
        match_ratio = float(len(matched_keywords) / len(job_keywords)) if job_keywords else 0.0

        # 3) Combined probability
        final_prob = combine_scores(model_prob, match_ratio)

        label = "relevant" if final_prob >= threshold else "irrelevant"

        summary = generate_summary(text)
        parsed = extract_structured_info(text)

        # Diagnostics
        parsed["job_keywords"] = job_keywords
        parsed["matched_keywords"] = matched_keywords
        parsed["keyword_match_ratio"] = match_ratio
        parsed["model_probability"] = model_prob
        parsed["final_probability"] = final_prob

        results.append(FileResult(
            filename=upload.filename,
            probability=final_prob,
            label=label,
            summary=summary,
            parsed=parsed
        ))

    if processed_files == 0:
        return AnalyzeResponse(
            total_files=0,
            relevant_count=0,
            irrelevant_count=0,
            results=[],
            job_keywords=job_keywords
        )

    relevant_count = sum(1 for r in results if r.label == "relevant")
    irrelevant_count = processed_files - relevant_count

    return AnalyzeResponse(
        total_files=processed_files,
        relevant_count=relevant_count,
        irrelevant_count=irrelevant_count,
        results=results,
        job_keywords=job_keywords
    )


# ------------------------------------------------------------
# 10) Frontend loader (/app -> index1.html)
# ------------------------------------------------------------
@app.get("/app", response_class=HTMLResponse)
def app_page():
    try:
        with open("index1.html", "r", encoding="utf-8") as f:
            html = f.read()
        return HTMLResponse(html)
    except FileNotFoundError:
        return HTMLResponse(
            "<h1>Frontend not found</h1><p>Place index1.html next to api1.py.</p>",
            status_code=404
        )


# ------------------------------------------------------------
# 11) Local run
# ------------------------------------------------------------
if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)


Overwriting api1.py


In [ ]:
%%writefile index1.html
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8" />
  <title>AI Resume Recommendation System</title>
  <meta name="viewport" content="width=device-width, initial-scale=1" />
  <style>
    body {
      font-family: system-ui, -apple-system, BlinkMacSystemFont, "Segoe UI", sans-serif;
      max-width: 1100px;
      margin: 20px auto;
      padding: 0 12px;
      line-height: 1.5;
      background: #f9fafb;
      color: #111827;
    }
    h1, h2, h3, h4 {
      font-weight: 600;
      margin-top: 0;
    }
    .card {
      background: #ffffff;
      border-radius: 10px;
      padding: 16px 20px;
      margin-bottom: 18px;
      box-shadow: 0 2px 8px rgba(15, 23, 42, 0.06);
    }
    label {
      font-weight: 600;
      margin-top: 10px;
      display: block;
      font-size: 0.92rem;
    }
    textarea {
      width: 100%;
      min-height: 130px;
      padding: 8px;
      box-sizing: border-box;
      border-radius: 6px;
      border: 1px solid #cbd5f5;
      font-family: inherit;
      font-size: 0.95rem;
      resize: vertical;
    }
    input[type="number"], input[type="text"] {
      width: 100%;
      padding: 8px;
      border-radius: 6px;
      border: 1px solid #cbd5f5;
      font-size: 0.95rem;
      box-sizing: border-box;
    }
    input[type="file"] {
      margin-top: 6px;
    }
    button {
      padding: 8px 15px;
      border-radius: 6px;
      border: none;
      cursor: pointer;
      font-size: 0.9rem;
      background-color: #2563eb;
      color: white;
      margin-top: 10px;
      margin-right: 8px;
    }
    button.secondary {
      background-color: #6b7280;
    }
    button:disabled {
      opacity: 0.6;
      cursor: default;
    }
    .small-text {
      font-size: 0.85rem;
      color: #6b7280;
    }
    .stats-row {
      display: flex;
      flex-wrap: wrap;
      gap: 12px;
      margin-top: 10px;
    }
    .stat-box {
      flex: 1 1 150px;
      background: #eef2ff;
      border-radius: 8px;
      padding: 8px 10px;
      font-size: 0.9rem;
    }
    .stat-label {
      color: #6b7280;
      font-size: 0.8rem;
      text-transform: uppercase;
      letter-spacing: 0.05em;
    }
    .stat-value {
      font-size: 1.1rem;
      font-weight: 600;
    }
    table {
      width: 100%;
      border-collapse: collapse;
      margin-top: 10px;
      font-size: 0.9rem;
      background: #ffffff;
    }
    th, td {
      border: 1px solid #e5e7eb;
      padding: 6px 8px;
      text-align: left;
    }
    th {
      background: #f3f4f6;
    }
    .tag {
      display: inline-block;
      padding: 2px 8px;
      font-size: 0.75rem;
      border-radius: 999px;
      background: #e5e7eb;
      margin-right: 4px;
      margin-bottom: 4px;
    }
    .tag.relevant {
      background: #dcfce7;
      color: #166534;
    }
    .tag.irrelevant {
      background: #fee2e2;
      color: #991b1b;
    }
    .chip {
      display: inline-block;
      padding: 2px 8px;
      font-size: 0.8rem;
      border-radius: 999px;
      background: #e0f2fe;
      color: #075985;
      margin: 2px;
    }
    .chip.keyword {
      background: #fef3c7;
      color: #92400e;
    }
    .chip.keyword-matched {
      background: #bbf7d0;
      color: #166534;
    }
    .preview-box {
      max-height: 220px;
      overflow-y: auto;
      padding: 8px;
      border-radius: 6px;
      border: 1px solid #e5e7eb;
      background: #f9fafb;
      white-space: pre-wrap;
      font-size: 0.86rem;
    }
    .exp-item {
      margin-bottom: 4px;
      font-size: 0.86rem;
    }
    .contact-line {
      font-size: 0.88rem;
      margin: 2px 0;
    }
    .contact-label {
      font-weight: 600;
    }
    @media (max-width: 700px) {
      .stats-row {
        flex-direction: column;
      }
    }
  </style>
</head>
<body>
  <h1>AI Resume Recommendation System</h1>
  <p class="small-text">
    Upload multiple resumes (PDF, DOCX, TXT, CSV, image) and provide a job description.
    The system will score each resume, classify relevance, analyze keywords, and generate structured summaries.
  </p>

  <!-- Job description and threshold -->
  <div class="card">
    <h2>1. Job description</h2>
    <label for="jobText">Job description</label>
    <textarea id="jobText" placeholder="Paste the job description here..."></textarea>

    <label for="thresholdInput" style="margin-top:10px;">Relevance threshold (0–1)</label>
    <input id="thresholdInput" type="number" step="0.01" value="0.5" />

    <p class="small-text" style="margin-top:8px;">
      Resumes with probability ≥ threshold are labelled as <strong>relevant</strong>.
    </p>
  </div>

  <!-- File upload and run -->
  <div class="card">
    <h2>2. Upload resumes</h2>
    <p class="small-text">
      Supported formats: <strong>.pdf, .docx, .txt, .csv, .jpg, .jpeg, .png</strong>.
      You can upload many files at once.
    </p>
    <input id="fileInput" type="file" multiple />

    <button id="btnAnalyze" type="button">Analyze resumes</button>
    <span id="analyzeStatus" class="small-text"></span>

    <div id="summaryStats" class="stats-row" style="display:none;">
      <div class="stat-box">
        <div class="stat-label">Total files</div>
        <div class="stat-value" id="statTotal">0</div>
      </div>
      <div class="stat-box">
        <div class="stat-label">Relevant</div>
        <div class="stat-value" id="statRelevant">0</div>
      </div>
      <div class="stat-box">
        <div class="stat-label">Not relevant</div>
        <div class="stat-value" id="statIrrelevant">0</div>
      </div>
    </div>
  </div>

  <!-- Results table + details -->
  <div class="card">
    <h2>3. Results</h2>

    <div id="jobKeywordsBlock" style="display:none; margin-bottom:10px;">
      <h4>Job keywords (auto-extracted from description)</h4>
      <div id="jobKeywordsContainer"></div>
    </div>

    <div id="resultsContainer" class="small-text">
      No results yet. Upload resumes and click “Analyze resumes”.
    </div>

    <div id="detailsCard" style="margin-top:18px; display:none;">
      <h3>Candidate details</h3>
      <p class="small-text" id="detailsHeader"></p>

      <h4>AI Summary</h4>
      <div id="detailsSummary" class="preview-box"></div>

      <h4>Contact information</h4>
      <div id="detailsContact"></div>

      <h4>Skills (auto-extracted from resume)</h4>
      <div id="detailsSkills"></div>

      <h4>Keyword matching with this job</h4>
      <p class="small-text" id="detailsKeywordRatio"></p>
      <div>
        <div class="small-text" style="margin-top:4px;">Matched keywords in this resume:</div>
        <div id="detailsMatchedKeywords"></div>
      </div>

      <h4>Education</h4>
      <div id="detailsEducation"></div>

      <h4>Experience snippets</h4>
      <div id="detailsExperience"></div>

      <h4>Resume preview</h4>
      <div id="detailsPreview" class="preview-box"></div>
    </div>
  </div>

  <script>
    // same origin (ngrok/local) – backend mounted at /analyze_files
    const apiBase = "";

    const jobTextEl = document.getElementById("jobText");
    const thresholdInputEl = document.getElementById("thresholdInput");
    const fileInputEl = document.getElementById("fileInput");
    const btnAnalyze = document.getElementById("btnAnalyze");
    const analyzeStatusEl = document.getElementById("analyzeStatus");

    const summaryStatsEl = document.getElementById("summaryStats");
    const statTotalEl = document.getElementById("statTotal");
    const statRelevantEl = document.getElementById("statRelevant");
    const statIrrelevantEl = document.getElementById("statIrrelevant");

    const jobKeywordsBlockEl = document.getElementById("jobKeywordsBlock");
    const jobKeywordsContainerEl = document.getElementById("jobKeywordsContainer");

    const resultsContainerEl = document.getElementById("resultsContainer");

    const detailsCardEl = document.getElementById("detailsCard");
    const detailsHeaderEl = document.getElementById("detailsHeader");
    const detailsSummaryEl = document.getElementById("detailsSummary");
    const detailsContactEl = document.getElementById("detailsContact");
    const detailsSkillsEl = document.getElementById("detailsSkills");
    const detailsKeywordRatioEl = document.getElementById("detailsKeywordRatio");
    const detailsMatchedKeywordsEl = document.getElementById("detailsMatchedKeywords");
    const detailsEducationEl = document.getElementById("detailsEducation");
    const detailsExperienceEl = document.getElementById("detailsExperience");
    const detailsPreviewEl = document.getElementById("detailsPreview");

    let lastResults = [];
    let lastJobKeywords = [];

    function renderJobKeywords(keywords) {
      jobKeywordsContainerEl.innerHTML = "";
      if (!keywords || !keywords.length) {
        jobKeywordsBlockEl.style.display = "none";
        return;
      }
      keywords.forEach(kw => {
        const span = document.createElement("span");
        span.className = "chip keyword";
        span.textContent = kw;
        jobKeywordsContainerEl.appendChild(span);
      });
      jobKeywordsBlockEl.style.display = "block";
    }

    function renderResults(data) {
      lastResults = data.results || [];
      lastJobKeywords = data.job_keywords || [];

      renderJobKeywords(lastJobKeywords);

      if (!lastResults.length) {
        resultsContainerEl.innerHTML = "<p class='small-text'>No valid resumes processed.</p>";
        summaryStatsEl.style.display = "none";
        detailsCardEl.style.display = "none";
        return;
      }

      statTotalEl.textContent = data.total_files;
      statRelevantEl.textContent = data.relevant_count;
      statIrrelevantEl.textContent = data.irrelevant_count;
      summaryStatsEl.style.display = "flex";

      let html = `
        <table>
          <thead>
            <tr>
              <th>#</th>
              <th>Filename</th>
              <th>Probability</th>
              <th>Label</th>
              <th>Keyword match</th>
              <th>Action</th>
            </tr>
          </thead>
          <tbody>
      `;

      lastResults.forEach((r, idx) => {
        const pct = (r.probability * 100).toFixed(2);
        const labelClass = r.label === "relevant" ? "relevant" : "irrelevant";
        const ratio = r.parsed && typeof r.parsed.keyword_match_ratio === "number"
          ? (r.parsed.keyword_match_ratio * 100).toFixed(1) + "%"
          : "N/A";

        html += `
          <tr>
            <td>${idx + 1}</td>
            <td>${r.filename}</td>
            <td>${r.probability.toFixed(4)} (${pct}%)</td>
            <td><span class="tag ${labelClass}">${r.label}</span></td>
            <td>${ratio}</td>
            <td><button class="secondary" type="button" data-idx="${idx}">View details</button></td>
          </tr>
        `;
      });

      html += "</tbody></table>";
      resultsContainerEl.innerHTML = html;

      resultsContainerEl.querySelectorAll("button[data-idx]").forEach(btn => {
        btn.addEventListener("click", () => {
          const idx = parseInt(btn.getAttribute("data-idx"), 10);
          showDetails(idx);
        });
      });

      detailsCardEl.style.display = "none";
    }

    function showDetails(idx) {
      const r = lastResults[idx];
      if (!r) return;

      const pct = (r.probability * 100).toFixed(2);
      detailsHeaderEl.textContent =
        `${r.filename} – Match probability ${r.probability.toFixed(4)} (${pct}%), label: ${r.label}`;

      // Summary
      detailsSummaryEl.textContent = r.summary || "(No summary available)";

      // Contact info
      const p = r.parsed || {};
      detailsContactEl.innerHTML = `
        <p class="contact-line"><span class="contact-label">Name:</span> ${p.name || "Not detected"}</p>
        <p class="contact-line"><span class="contact-label">Email:</span> ${p.email || "Not detected"}</p>
        <p class="contact-line"><span class="contact-label">Phone:</span> ${p.phone || "Not detected"}</p>
        <p class="contact-line"><span class="contact-label">LinkedIn:</span> ${p.linkedin || "Not detected"}</p>
        <p class="contact-line"><span class="contact-label">GitHub:</span> ${p.github || "Not detected"}</p>
      `;

      // Resume skills
      detailsSkillsEl.innerHTML = "";
      if (p.skills && p.skills.length) {
        p.skills.forEach(skill => {
          const span = document.createElement("span");
          span.className = "chip";
          span.textContent = skill;
          detailsSkillsEl.appendChild(span);
        });
      } else {
        detailsSkillsEl.innerHTML = "<p class='small-text'>No skills detected.</p>";
      }

      // Keyword match details
      const matchRatio = typeof p.keyword_match_ratio === "number"
        ? (p.keyword_match_ratio * 100).toFixed(1) + "%"
        : "N/A";
      detailsKeywordRatioEl.textContent =
        `Matched ${p.matched_keywords ? p.matched_keywords.length : 0} of ${lastJobKeywords.length} job keywords (${matchRatio}).`;

      detailsMatchedKeywordsEl.innerHTML = "";
      if (p.matched_keywords && p.matched_keywords.length) {
        p.matched_keywords.forEach(kw => {
          const span = document.createElement("span");
          span.className = "chip keyword-matched";
          span.textContent = kw;
          detailsMatchedKeywordsEl.appendChild(span);
        });
      } else {
        detailsMatchedKeywordsEl.innerHTML = "<p class='small-text'>No job keywords detected in this resume.</p>";
      }

      // Education
      detailsEducationEl.innerHTML = "";
      if (p.education && p.education.length) {
        p.education.forEach(ed => {
          const div = document.createElement("div");
          div.className = "exp-item";
          div.textContent = "• " + ed;
          detailsEducationEl.appendChild(div);
        });
      } else {
        detailsEducationEl.innerHTML = "<p class='small-text'>No education information detected.</p>";
      }

      // Experience snippets
      detailsExperienceEl.innerHTML = "";
      if (p.experience && p.experience.length) {
        p.experience.forEach(sn => {
          const div = document.createElement("div");
          div.className = "exp-item";
          div.textContent = "• " + sn;
          detailsExperienceEl.appendChild(div);
        });
      } else {
        detailsExperienceEl.innerHTML = "<p class='small-text'>No explicit experience snippets detected.</p>";
      }

      // Preview
      detailsPreviewEl.textContent = p.preview || "(No preview available)";
      detailsCardEl.style.display = "block";
    }

    btnAnalyze.onclick = async () => {
      resultsContainerEl.innerHTML = "";
      summaryStatsEl.style.display = "none";
      detailsCardEl.style.display = "none";
      jobKeywordsBlockEl.style.display = "none";
      analyzeStatusEl.textContent = " Running analysis...";
      btnAnalyze.disabled = true;

      const jobText = jobTextEl.value.trim();
      if (!jobText) {
        alert("Please paste a job description.");
        analyzeStatusEl.textContent = "";
        btnAnalyze.disabled = false;
        return;
      }

      let threshold = parseFloat(thresholdInputEl.value);
      if (isNaN(threshold) || threshold < 0 || threshold > 1) {
        alert("Threshold must be between 0 and 1.");
        analyzeStatusEl.textContent = "";
        btnAnalyze.disabled = false;
        return;
      }

      const files = Array.from(fileInputEl.files || []);
      if (!files.length) {
        alert("Please select at least one resume file.");
        analyzeStatusEl.textContent = "";
        btnAnalyze.disabled = false;
        return;
      }

      const formData = new FormData();
      formData.append("job", jobText);
      formData.append("threshold", String(threshold));
      files.forEach(f => formData.append("files", f));

      try {
        const resp = await fetch(apiBase + "/analyze_files", {
          method: "POST",
          body: formData
        });
        if (!resp.ok) {
          const txt = await resp.text();
          console.error("API error response:", txt);
          analyzeStatusEl.textContent = " API error.";
          btnAnalyze.disabled = false;
          return;
        }
        const data = await resp.json();
        renderResults(data);
        analyzeStatusEl.textContent = " Done.";
      } catch (err) {
        console.error(err);
        analyzeStatusEl.textContent = " Error contacting API.";
      } finally {
        btnAnalyze.disabled = false;
      }
    };
  </script>
</body>
</html>


Overwriting index1.html


In [ ]:
pip install fastapi uvicorn pdfplumber python-docx pillow pytesseract transformers accelerate joblib pandas


In [ ]:
!
!kill -9 $(lsof -t -i:8000)
!pip install  pyngrok python-docx pdfplumber pillow pytesseract transformers accelerate

from pyngrok import ngrok
import uvicorn, threading
from api1 import app

ngrok.set_auth_token("2y6jEVgXHUKILK47H0WNxTpfs0v_7Qv1nL2HkEoECKhXo4pox")

public_url = ngrok.connect(8000, "http").public_url
print("PUBLIC URL:", public_url)
print("Docs:", public_url + "/docs")
print("App:", public_url + "/app")

def start():
    uvicorn.run(app, host="0.0.0.0", port=8000)

threading.Thread(target=start, daemon=True).start()

kill: usage: kill [-s sigspec | -n signum | -sigspec] pid | jobspec ... or kill -l [sigspec]


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cpu


PUBLIC URL: https://aa467988760c.ngrok-free.app
Docs: https://aa467988760c.ngrok-free.app/docs
App: https://aa467988760c.ngrok-free.app/app


In [ ]:
!kill -9 $(lsof -t -i:8000)


kill: usage: kill [-s sigspec | -n signum | -sigspec] pid | jobspec ... or kill -l [sigspec]
